In [1]:
import tensorflow as tf
import utils.dataset as dset
import tensorflow_addons as tfa

# Baseline CNN model
Let's define a baseline CNN model using well known best practices.

The first layer is the input layer with shape (None, 48, 48, 3) because in the preprocessing the original patch_camelyon images (96x96x3) are:
1. Rescaled to (64, 64, 3).
2. Random cropped to (48, 48, 3).

In [2]:
# Input layer
input = tf.keras.layers.Input((48, 48, 3))

In [3]:
x = tf.keras.layers.Conv2D(64, 5, activation=None)(input)
x = tf.keras.layers.LeakyReLU()(x)
x = tfa.layers.InstanceNormalization(axis=3,
                                     center=True,
                                     scale=True,
                                     beta_initializer="random_uniform",
                                     gamma_initializer="random_uniform")(x)

In [4]:
x = tf.keras.layers.Conv2D(128, 3, activation=None)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tfa.layers.InstanceNormalization(axis=3,
                                     center=True,
                                     scale=True,
                                     beta_initializer="random_uniform",
                                     gamma_initializer="random_uniform")(x)
x = tf.keras.layers.MaxPooling2D()(x)

In [5]:
x = tf.keras.layers.Conv2D(128, 3, activation=None)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tfa.layers.InstanceNormalization(axis=3,
                                     center=True,
                                     scale=True,
                                     beta_initializer="random_uniform",
                                     gamma_initializer="random_uniform")(x)
x = tf.keras.layers.MaxPooling2D()(x)

In [6]:
x = tf.keras.layers.Conv2D(256, 3, activation=None, padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tfa.layers.InstanceNormalization(axis=3,
                                     center=True,
                                     scale=True,
                                     beta_initializer="random_uniform",
                                     gamma_initializer="random_uniform")(x)

In [7]:
x = tf.keras.layers.Conv2D(256, 3, activation=None, padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tfa.layers.InstanceNormalization(axis=3,
                                     center=True,
                                     scale=True,
                                     beta_initializer="random_uniform",
                                     gamma_initializer="random_uniform")(x)
x = tf.keras.layers.GlobalMaxPooling2D()(x)

In [8]:
x = tf.keras.layers.Dense(512, activation=None)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(512, activation=None)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Dropout(0.5)(x)

In [10]:
output = tf.keras.layers.Dense(1)(x)

In [11]:
baseline = tf.keras.Model(input, output)

In [12]:
baseline.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 44, 44, 64)        4864      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 44, 44, 64)        0         
_________________________________________________________________
instance_normalization (Inst (None, 44, 44, 64)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 42, 42, 128)       0         
_________________________________________________________________
instance_normalization_1 (In (None, 42, 42, 128)       256   

In [13]:
ds_train, ds_valid = dset.load_train_val_dataset()
ds_train = dset.preprocess_train(ds_train, batch_size=64)
ds_valid = dset.preprocess_validation(ds_valid)

In [14]:
baseline.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics='binary_accuracy')

In [15]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3),
    tf.keras.callbacks.EarlyStopping(min_delta=1e-2, patience=5)
]

In [16]:
baseline.fit(ds_train, epochs=50,
             validation_data=ds_valid, callbacks=callbacks)


Epoch 1/50
4096/4096 [==============================] - 126s 30ms/step - loss: 0.4432 - binary_accuracy: 0.7824 - val_loss: 0.3908 - val_binary_accuracy: 0.8155
Epoch 2/50
4096/4096 [==============================] - 123s 30ms/step - loss: 0.3542 - binary_accuracy: 0.8379 - val_loss: 0.4307 - val_binary_accuracy: 0.7693
Epoch 3/50
4096/4096 [==============================] - 123s 30ms/step - loss: 0.3208 - binary_accuracy: 0.8576 - val_loss: 0.3807 - val_binary_accuracy: 0.8132
Epoch 4/50
4096/4096 [==============================] - 123s 30ms/step - loss: 0.2974 - binary_accuracy: 0.8698 - val_loss: 0.3832 - val_binary_accuracy: 0.8291
Epoch 5/50
4096/4096 [==============================] - 124s 30ms/step - loss: 0.2807 - binary_accuracy: 0.8792 - val_loss: 0.3738 - val_binary_accuracy: 0.8282
Epoch 6/50
4096/4096 [==============================] - 124s 30ms/step - loss: 0.2662 - binary_accuracy: 0.8863 - val_loss: 0.3544 - val_binary_accuracy: 0.8469
Epoch 7/50
4096/4096 [============